In [ ]:
import hvplot.xarray

import matplotlib.pyplot as plt
import proplot as pplt
import numpy as np
import pandas as pd
import panel as pn
import seaborn as sns
import xarray as xr
from numpy.fft import fft, fftfreq, fftshift, ifft
from scipy import signal, stats


from troposim import igram_sim, turbulence
from apertools import lowess, plotting, utils

pn.extension()
# pplt.rc.update({"subplots.share": False, "subplots.span": False})


sns.set_style(style="white")
# plotting.set_style(size=16)


RNG = np.random.default_rng()

%matplotlib inline
# %matplotlib widget

%load_ext autoreload
%autoreload 2

In [ ]:
# sns.set_style?

# Chapter: Improved Robust Long-term Time Series Methods

1. Limitations of stacking to long time series
2. Robust Time Series Methods
    1. Regularization (Supplement from GRL)
    1. LPF by triangle filiter 
    1. LOWESS smoothing
3. Synthetic Example
    1. frequency response of triangle
    1. response of LOWESS
4. 7 Year Time Series for the Permian Basin
    1. Comparison to GPS
    2. Anthropogenic Caused Deformation Patterns

In [ ]:
r = 100
b = 2.7
N = 10
shape = (100, 100)
ny, nx = shape

igm_g = igram_sim.IgramMaker(
    num_days=N,
    shape=shape,
    distribution="normal",
    resolution=r,
    p0_default=100,
    to_cm=True,
)
igm_u = igram_sim.IgramMaker(
    num_days=N,
    shape=shape,
    distribution="uniform",
    resolution=r,
    p0_default=100,
    to_cm=True,
)

sar_stack_u, sar_date_list = igm_u.make_sar_stack(beta=b)
sar_stack_g, _ = igm_g.make_sar_stack(beta=b)

In [ ]:
sar_stack_g.max(), sar_stack_g.min()

In [ ]:
plt.figure()
plt.imshow(sar_stack_u[0])
plt.colorbar()

In [ ]:
s = sar_stack_u[sar_stack_u.max(axis=(1, 2)).argmax()]

plt.figure()
plt.imshow(s)
plt.colorbar()

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(8, 3), sharex=True, sharey=True, squeeze=False)
axes = axes.ravel()

# axes[0].hist(sar_stack1.ravel())
# axes[1].hist(sar_stack2.ravel())
ax = axes[0]
for s in sar_stack_g:
    sns.kdeplot(x=s.ravel(), ax=ax)
ax.set_title("Gaussian")

ax = axes[1]
for s in sar_stack_u:
    sns.kdeplot(x=s.ravel(), ax=ax)
ax.set_title("Uniform")

In [ ]:
fig, ax = plt.subplots(figsize=(5, 5), sharex=True, sharey=True)

sns.kdeplot(x=sar_stack_g.ravel(), ax=ax, label="Gaussian")
sns.kdeplot(x=sar_stack_u.ravel(), ax=ax, label="Uniform")
ax.legend()

# Add defo 

In [ ]:
p0_rv = "gamma"
p0_params = {"scale": 50, "a": 2, "loc": 2}

plt.figure()
xx = np.linspace(0, 250)

plt.plot(xx, stats.gamma(**p0_params).pdf(xx))
plt.xlabel("p0")

In [ ]:
defo_rates = [2, -2]
# num_days = 80 # 2.6 years
num_days = 91  # 3 years
r = 200  # resolution, m
# b = 2.7

shape = (300, 300)

igm = igram_sim.IgramMaker(
    num_days=num_days, shape=shape, resolution=r, to_cm=True, defo_rates=defo_rates
)

In [ ]:
# Draw the daily powers from an exponential

seed = 1
beta_arr_slopes = stats.norm.rvs(
    loc=-2.6, scale=0.1, size=(num_days,), random_state=seed
)
beta_arr = np.vstack((-9.7 * np.ones(num_days), beta_arr_slopes)).T

sar_stack, sar_date_list = igm.make_sar_stack(
    beta=beta_arr, p0_params=p0_params, p0_rv=p0_rv, seed=seed
)
dd = igm.make_defo_stack(sigma=15)

In [ ]:
sns.histplot(np.max(np.abs(sar_stack), axis=(1, 2)), bins=20)

In [ ]:
plt.figure()
plt.imshow(igm.sar_stack[4])
plt.colorbar()

In [ ]:
plt.figure()
plt.imshow(dd[4])
plt.colorbar()

In [ ]:
dates = pd.to_datetime(igm.sar_date_list)
dd_xr = utils.stack_to_xr(dd, z_coords=dates, dims=("date", "y", "x"))
noise_xr = utils.stack_to_xr(igm.sar_stack, z_coords=dates, dims=("date", "y", "x"))

In [ ]:
# dd_xr
# noise_xr

In [ ]:
plotting.hvplot_stack(dd_xr, x="x", y="y")

In [ ]:
plotting.hvplot_stack(noise_xr, x="x", y="y")

# Time series of center of deformation

In [ ]:
noise = igm.sar_stack.copy()
noise = noise - noise[:, 5, 5][:, None, None]
# Make the day1 noise non-trivial
# day1max = np.abs(noise[0]).max()

# noisemids = np.abs(noise[:, ny//2, nx//2])
noisemids = noise[:, ny // 2, nx // 2]
# mid_idx = np.argmax(noisemids)
mid_idx = np.argsort(noisemids)[-3]  # get 3rd largest, not max
img1, img2 = noise[0], noise[mid_idx]
noise[0] = img2
noise[mid_idx] = img1

# newmax = 2
# noise[0] = (noise[0] / day1max) * newmax
# save for later
day1noise = noise[0].copy()
# Subtract the day 1 from the rest:
noise = noise - noise[0][None, :, :]

defo = igm.defo_stack.copy()
defo_plus_noise = noise + defo
dn_xr = utils.stack_to_xr(defo_plus_noise, z_coords=dates, dims=("date", "y", "x"))

In [ ]:
day1noise.max(), day1noise.min()

In [ ]:
nmax = np.abs(day1noise).max()
plt.imshow(day1noise, cmap="seismic_wide_y_r", vmax=nmax, vmin=-nmax)
plt.colorbar()

In [ ]:
plotting.hvplot_stack(dn_xr, x="x", y="y")

In [ ]:
dn_xr.std(dim=("date")).shape

In [ ]:
plt.figure()
dn_xr.std(dim=("date")).plot.imshow()
plt.figure()
noise_xr.std(dim=("date")).plot.imshow()

In [ ]:
ts_noisy = dn_xr.sel(x=nx // 2, y=ny // 2).copy()
ts_truth = dd_xr.sel(x=nx // 2, y=ny // 2).copy()

sigma = 0.7
ts_white_noise = ts_truth.copy() + RNG.normal(scale=sigma, size=len(ts_noisy))
ts_white_noise -= ts_white_noise[0]

In [ ]:
# stats.kstest?

In [ ]:
(
    stats.kstest(ts_noisy.data - ts_truth.data, stats.norm.cdf),
    stats.kstest(stats.norm.rvs(size=100), stats.norm.cdf),
    stats.kstest(ts_white_noise.data - ts_truth.data, stats.norm.cdf),
)

In [ ]:
fig, ax = plt.subplots()
ts_white_noise.plot(label="white noise", ax=ax)
ts_noisy.plot(label="turbulence", ax=ax)
ax.legend()

In [ ]:
fig, ax = plt.subplots(figsize=(7, 4))
sns.kdeplot(stats.norm.rvs(size=len(ts_noisy), scale=sigma), label="norm.rvs")
sns.kdeplot(ts_white_noise.data - ts_truth.data, label="white")
sns.kdeplot(ts_noisy.data - ts_truth.data, label="turb")
ax.legend()

# LOWESS Algorithm Demonstration

## LOWESS windows only

In [ ]:
fig, ax = pplt.subplots(refwidth=5, refheight=2, grid=True)
xx = np.linspace(0, 1, 100)
idx = 20

for frac, fstr in zip([1 / 6, 1 / 3, 1 / 2], ["1/6", "1/3", "1/2"]):
    w = lowess.demo_window(xx, frac=frac)
    ax.plot(xx, w[:, idx], label=f"$\gamma =${fstr}", lw=3)

# ax.legend(loc='upper left')
ax.legend(loc="upper right", ncols=1)

ax.set_ylabel("$\mathbf{w}(t_i)$")

# ax.set_ylim((None, .1))
ax.set_xticks([idx / 100], ["$t_i$"])
ax.set_xlabel("$\mathbf{t}$")

In [ ]:
# f = 1/2
# x = xx

# n = len(x)
# r = int(np.ceil(f * n))
# h = [np.sort(np.abs(x - x[i]))[r] for i in range(n)]
# w = np.clip(np.abs((x[:, None] - x[None, :]) / h), 0.0, 1.0)
# w = (1 - w ** 3) ** 3
# plt.figure(); plt.plot(w[:, 40])

In [ ]:
fig.savefig("../figures/chapter5-lowess/figure1-window.pdf")

## LOWESS windows, time series example

In [ ]:
dn_xr_noisy = dn_xr.copy()

# Make something near the demo index noisy
idx = 20

dn_xr_noisy[idx + 7] += (dn_xr_noisy[idx + 7] + 5)
dn_xr_noisy[idx + 4] += (-dn_xr_noisy[idx + 4] + 6.5)
dn_xr_noisy[59] += (-dn_xr_noisy[59] - 3.7)
dn_xr_noisy[60] += (-dn_xr_noisy[60] - 2.9)

ts_noisy = dn_xr_noisy.sel(x=nx // 2, y=ny // 2).copy()
# ts_noisy[idx + 7] = 5
# ts_noisy[idx + 4] = 6.5
# ts_noisy[59] = -3.7
# ts_noisy[60] = -2.9

x = lowess.date2num(dn_xr.date.values)
x = (x - x[0]) / 365.25
# x



In [ ]:
plt.figure()
plt.plot(dn_xr_noisy[:, nx//2, ny//2])
plt.plot(dn_xr[:, nx//2, ny//2])

In [ ]:
pplt.rc["cycle"] = "colorblind"
pplt.rc["grid.alpha"] = 0.4

In [ ]:
fig, ax = pplt.subplots(refwidth=5, refheight=2, grid=True)
ax2 = ax.twinx()

# lon, lat = gps.station_lonlat("TXKM")
# ax2.scatter(igm.sar_date_list, ts_noisy.data, label=r"$\phi$", marker='.', lw=1, color="gold", s=100)
# Using the spiked
ax2.scatter(
    igm.sar_date_list, y, label=r"$\phi$", marker=".", lw=1, color="gold", s=100
)


years_smooth = [0.5, 1, 1.5]
for year in years_smooth:
    w = lowess.demo_window(x, min_x_weighted=year)
    ax.plot(igm.sar_date_list, w[:, idx], label=f"{year} year", lw=3)

# ax.legend(loc='upper left')
ax.legend(loc="upper right", ncols=1)
# ax2.legend()
# ax.set_title("Initial window weight for each LOWESS iteration")
ax2.set_title("")
ax2.set_ylabel(f"$\phi$ [cm]")

# ax2.set_ylim((-7, 7))
ax.set_ylabel("$\mathbf{w}(t_i)$")

In [ ]:
fig.savefig("../figures/chapter5-lowess/figure1-window-ts.pdf")

In [ ]:
# fig, ax = plt.subplots()
# ax2 = ax.twinx()

# # lon, lat = gps.station_lonlat("TXKM")
# ts_noisy.plot(ax=ax2, label="insar", marker='.', lw=1, color='gold')


# years_smooth = [0.5, 1, 1.5]
# for year in years_smooth:
#     w = lowess.demo_window(x, min_x_weighted=year)
#     ax.plot(igm.sar_date_list, w[50], label=f"{year} year", lw=3)

# ax.legend(loc='upper left')
# # ax2.legend()
# ax.set_title("Initial window weight for each LOWESS iteration")
# ax2.set_title("")
# ax2.set_ylabel("InSAR [cm]")

# # ax2.set_ylim((-7, 7))
# ax.set_ylabel("$\mathbf{w}(t_i)$")

## Time series only plot, plus image of deformation

In [ ]:
ts_arr = [ts_truth, ts_noisy]
# labels = ['Noisy', 'Truth']
labels = ["$\mathbf{d}$", "$\mathbf{\phi}$"]
colors = ["C1", "C0"]

fig, axes = pplt.subplots([[1, 2], [3, 3]], abc="(a)")  # , figsize=(6, 3))


ax.set_ylabel("[cm]")
# ax.set_title("Triangle-window smoothing")
# ax.grid()

ax.legend()
defo_img = dd_xr[len(dd_xr) // 2]
img_date = dd_xr.indexes["date"][len(dd_xr) // 2]
defo_noise_img = dn_xr[len(dd_xr) // 2]
vm = np.abs(defo_noise_img).max()


ax = axes[0]
# axim = ax.imshow(noise_xr[len(dd_xr)//2], cmap="RdBu_r", vmax=vm, vmin=-vm)
axim = ax.imshow(defo_img, cmap="RdBu_r", vmax=vm, vmin=-vm)
ax.colorbar(axim, loc="r", label="[cm]")
ax.format(grid=False, title=f"{labels[0]}: {str(img_date.date())}")
ax.plot(nx // 2, ny // 2, "X", color=colors[0])


ax = axes[1]
axim = ax.imshow(defo_noise_img, cmap="RdBu_r", vmax=vm, vmin=-vm)
ax.colorbar(axim, loc="r", label="[cm]")
ax.format(grid=False, title=f"{labels[1]}: {str(img_date.date())}")
ax.plot(nx // 2, ny // 2, "X", color=colors[1])

ax = axes[2]

dts = ts_noisy.date.values
# for tt, label, color in zip(ts_arr, labels, colors):
ax.plot(dts, ts_arr[0], lw=3, label=labels[0], color=colors[0])
ax.plot(dts, ts_arr[1], lw=1.5, label=labels[1], marker=".", color=colors[1])
ax.set_ylabel("[cm]")
ax.legend(loc="ur")

ylim = ax.get_ylim()  # Keep the same limits, but make a vert line for whole span
ax.vlines(img_date, ylim[0], ylim[1], linestyle="--", color="gray", alpha=0.5)
ax.set_ylim(ylim)

In [ ]:
fig.savefig("../figures/chapter5-lowess/figure2-demo-data.pdf")

### Color tests

In [ ]:
# First Iteration
from matplotlib.colors import LinearSegmentedColormap, to_rgb, to_rgba


def get_alpha_by_value(alpha_arr, color="C1"):
    r, g, b = to_rgb(color)
    # r, g, b, _ = to_rgba(color)
    return [(r, g, b, alpha) for alpha in alpha_arr]


get_alpha_by_value([0.5, 0.2, 1])

In [ ]:
# fig, axs = pplt.show_cycles(rasterized=True)

In [ ]:
# LinearSegmentedColormap.from_list('', [to_rgba("C2"), to_rgba("grey")], N)

In [ ]:
# pplt.rc['cycle'] = 'tab10'

In [ ]:
cmap = LinearSegmentedColormap.from_list("", [to_rgba("grey"), to_rgba("gold")], N)
cmap

## LOWESS window color shading

In [ ]:
# idx = 20
# w = lowess.demo_window(x, min_x_weighted=1.0)
# window = w[:, idx]

# # color_arr = get_alpha_by_value(window)
# # map from gray at 0 to green at max
# N = 256
# # cmap = LinearSegmentedColormap.from_list('', [to_rgba("C6"), to_rgba("C8")], N)
# colors = cmap(window)

# # fig, ax = pplt.subplots(figsize=(8, 5))
# fig, ax = pplt.subplots(refwidth=5, refheight=2)

# ax2 = ax.twinx()

# # lon, lat = gps.station_lonlat("TXKM")
# aa = ax2.scatter(igm.sar_date_list, ts_noisy.data, label="insar", marker='.', lw=1, color=colors, s=100)


# ax.plot(igm.sar_date_list, window, lw=3)

# # ax.legend(loc='upper left')
# # ax2.legend()
# # ax.set_title("Initial window weight for initial LOWESS iteration")
# ax2.set_title("")
# ax2.set_ylabel("InSAR [cm]")

# # ax2.set_ylim((-7, 7))
# ax.set_ylabel("$\mathbf{w}(t_i)$")

# # cmap

## LOWESS First iteration slope fit

In [ ]:
idx = 20
w = lowess.demo_window(x, min_x_weighted=1.0)
beta = lowess.demo_fit(x, ts_noisy.data, w, idx)
x_slice = x[idx - 10 : idx + 10]
yest = beta[0] + beta[1] * x_slice
# yest

In [ ]:
from matplotlib.colors import LinearSegmentedColormap, to_rgba

w = lowess.demo_window(x, min_x_weighted=1.0)
window = w[:, idx]

# color_arr = get_alpha_by_value(window)
# map from gray at 0 to green at max
N = 256
# cmap = LinearSegmentedColormap.from_list('', [to_rgba("C6"), to_rgba("C8")], N)
colors = cmap(window)

# fig, ax = plt.subplots(figsize=(8, 5))
fig, ax = pplt.subplots(refwidth=5, refheight=2)
ax2 = ax.twinx()

fit_color = "C1"
# lon, lat = gps.station_lonlat("TXKM")
ax2.scatter(
    igm.sar_date_list,
    ts_noisy.data,
    label=r"$\phi$",
    marker=".",
    lw=1,
    color=colors,
    s=100,
)
ax2.plot(igm.sar_date_list[idx - 10 : idx + 10], yest, lw=6, color=fit_color)

ax.plot(igm.sar_date_list, window, lw=3)

# ax.legend(loc='upper left')
# ax2.legend()
ax.set_title("Initial fit")
ax2.set_title("")
ax2.set_ylabel("InSAR [cm]")

# ax2.set_ylim((-7, 7))
ax.set_ylabel("$\mathbf{w}(t_i)$")

In [ ]:
resid = lowess.demo_residual(x, ts_noisy.data, w, idx)
beta_2 = lowess.demo_fit(x, ts_noisy.data, w, idx, delta=resid)
yest_2 = beta_2[0] + beta_2[1] * x_slice
yest_2

In [ ]:
from matplotlib.colors import LinearSegmentedColormap, to_rgba

w = lowess.demo_window(x, min_x_weighted=1.0)
window = w[:, idx]
window2 = resid * window

# color_arr = get_alpha_by_value(window)
# map from gray at 0 to green at max
# N = 256
# cmap = LinearSegmentedColormap.from_list('', [to_rgba("C6"), to_rgba("C8")], N)
colors2 = cmap(window2)

# fig, ax = plt.subplots(figsize=(8, 5))
fig, ax = pplt.subplots(refwidth=5, refheight=2)

ax2 = ax.twinx()

fit_color = "C1"
fit_color2 = "C2"
# lon, lat = gps.station_lonlat("TXKM")
ax2.scatter(
    igm.sar_date_list,
    ts_noisy.data,
    label="$\phi$",
    marker=".",
    lw=1,
    color=colors2,
    s=100,
)
ax2.plot(igm.sar_date_list[idx - 10 : idx + 10], yest, lw=5, color=fit_color, alpha=0.5)
ax2.plot(igm.sar_date_list[idx - 10 : idx + 10], yest_2, lw=6, color=fit_color2)

# ax.plot(igm.sar_date_list, window, lw=3)
ax.plot(igm.sar_date_list, window2, lw=3)

# ax.legend(loc='upper left')
# ax2.legend()
# ax.set_title("Second iteration fit")
ax2.set_title("")
ax2.set_ylabel("\phi [cm]")

# ax2.set_ylim((-7, 7))
ax.set_ylabel("$\mathbf{w}(t_i)$")

## Combined first 2 iterations

In [ ]:
# Recalculate the 2 estimates
idx = 20

w = lowess.demo_window(x, min_x_weighted=1.0)
beta = lowess.demo_fit(x, ts_noisy.data, w, idx)
x_slice = x[idx - 10 : idx + 10]
yest = beta[0] + beta[1] * x_slice


resid = lowess.demo_residual(x, ts_noisy.data, w, idx)
beta_2 = lowess.demo_fit(x, ts_noisy.data, w, idx, delta=resid)
yest_2 = beta_2[0] + beta_2[1] * x_slice
yest_2

In [ ]:
y = ts_noisy.data
y_iter1 = lowess.lowess_pixel(y, x, frac=1 / 3, n_iter=1)
y_iter2 = lowess.lowess_pixel(y, x, frac=1 / 3, n_iter=2)


w = lowess.demo_window(x, min_x_weighted=1.0)
window = w[:, idx]


N = 256
colors = cmap(window)

# fig, axes = pplt.subplots(refwidth=4, refheight=1.2, nrows=3, abc="(a)")
fig, axes = pplt.subplots(ncols=2, nrows=2, abc="(a)", sharey=False)
# fig, ax = plt.subplots(figsize=(8, 5))

ax1 = axes[0]
ax2 = ax1.twinx()

fit_color = "C1"
ax1.scatter(
    igm.sar_date_list,
    ts_noisy.data,
    label=r"$\phi$",
    marker=".",
    lw=1,
    color=colors,
    s=100,
)
ax1.plot(igm.sar_date_list[idx - 10 : idx + 10], yest, lw=6, color=fit_color)
ax1.set_ylabel("$\phi$ [cm]")

ax2.plot(igm.sar_date_list, window, lw=3, color=fit_color)
# ax2.set_ylim((-7, 7))
ax2.set_ylabel("$\mathbf{w}(t_i)$")

######################################

ax = axes[1]

fit_color = "C1"
ax.scatter(
    igm.sar_date_list, ts_noisy.data, marker=".", lw=1, color="gray", s=100
)  # , label=r"$\phi$")
ax.plot(igm.sar_date_list, y_iter1, lw=4, color=fit_color, label="1 iters")


ax.set_ylabel("$\phi$ [cm]")


###############################
window2 = resid * window
colors2 = cmap(window2)

ax1 = axes[2]
ax2 = ax1.twinx()

fit_color = "C1"
fit_color2 = "C2"

alpha = 0.7
ax1.scatter(
    igm.sar_date_list,
    ts_noisy.data,
    label="$\phi$",
    marker=".",
    lw=1,
    color=colors2,
    s=100,
)
ax1.plot(
    igm.sar_date_list[idx - 10 : idx + 10], yest, lw=5, color=fit_color, alpha=alpha
)
ax1.plot(igm.sar_date_list[idx - 10 : idx + 10], yest_2, lw=6, color=fit_color2)
ax1.set_ylabel("$\phi$ [cm]")

ax2.plot(igm.sar_date_list, window2, lw=3, color=fit_color2)
# ax2.set_ylim((-7, 7))
ax2.set_ylabel("$\mathbf{w}(t_i)$")


######################################
ax = axes[3]

fit_color = "C1"
ax.scatter(
    igm.sar_date_list, ts_noisy.data, marker=".", lw=1, color="gray", s=100
)  # , label=r"$\phi$")
ax.plot(igm.sar_date_list, y_iter1, lw=4, color=fit_color, alpha=alpha, label="1 iters")
ax.plot(igm.sar_date_list, y_iter2, lw=3, color=fit_color2, label="2 iters")


ax.set_title("")
ax.set_ylabel("$\phi$ [cm]")
ax.legend(loc="lower left", ncols=1)

In [ ]:
fig.savefig("../figures/chapter5-lowess/figure3-fits.pdf")

In [ ]:
# from matplotlib.colors import LinearSegmentedColormap, to_rgba

# w = lowess.demo_window(x, min_x_weighted=1.0)
# window = w[:, idx]

# # color_arr = get_alpha_by_value(window)
# # map from gray at 0 to green at max
# N = 256
# cmap = LinearSegmentedColormap.from_list('', [to_rgba("C6"), to_rgba("C8")], N)
# colors = cmap(window)

# fig, ax = plt.subplots(figsize=(8, 5))
# ax2 = ax.twinx()

# # lon, lat = gps.station_lonlat("TXKM")
# ax2.scatter(igm.sar_date_list, ts_noisy.data, label="insar", marker='.', lw=1, color=colors, s=100)
# ax2.plot(igm.sar_date_list[idx-10:idx+10], yest, lw=5, color='gray')
# ax2.plot(igm.sar_date_list[idx-10:idx+10], yest_2, lw=6)

# ax.plot(igm.sar_date_list, window, lw=3)

# ax.legend(loc='upper left')
# # ax2.legend()
# ax.set_title("Second iteration fit")
# ax2.set_title("")
# ax2.set_ylabel("InSAR [cm]")

# # ax2.set_ylim((-7, 7))
# ax.set_ylabel("$\mathbf{w}(t_i)$")

In [ ]:
y_iter2_zeroed = y_iter2 - y_iter2[0]
ts_arr = [ts_truth, ts_noisy, y_iter2_zeroed]
# labels = ['Noisy', 'Truth']
labels = ["$\mathbf{d}$", "$\mathbf{\phi}$", "$\mathbf{\hat{\phi}}$"]
colors = ["C1", "C0", "C2"]
# [[1, 2], [3, 3]],
fig, axes = pplt.subplots(abc="(a)", figsize=(6, 3))

ax = axes[0]
dts = ts_noisy.date.values
# for tt, label, color in zip(ts_arr, labels, colors):
ax.plot(dts, ts_truth, lw=4, label=labels[0], color=colors[0])
ax.plot(dts, ts_noisy, lw=2, label=labels[1], marker=".", color=colors[1])
ax.plot(dts, y_iter2_zeroed, lw=4, label=labels[2], color=colors[2])

ax.set_ylabel("[cm]")
ax.legend(loc="ur")

## Plot fits from first 2 iterations

In [ ]:
y = ts_noisy.data
y_iter1 = lowess.lowess_pixel(y, x, frac=1 / 3, n_iter=1)
y_iter2 = lowess.lowess_pixel(y, x, frac=1 / 3, n_iter=2)


fig, axes = pplt.subplots(refwidth=3.5, refheight=1.5, nrows=1, abc="(a)")


ax = axes[0]

fit_color = "C1"
ax.scatter(
    igm.sar_date_list, ts_noisy.data, marker=".", lw=1, color="gray", s=100
)  # , label=r"$\phi$")
ax.plot(igm.sar_date_list, y_iter1, lw=4, color=fit_color, label="1 iters")
ax.plot(igm.sar_date_list, y_iter2, lw=3, color=fit_color2, label="2 iters")


ax.set_title("")
ax.set_ylabel("$\phi$ [cm]")
ax.legend(loc="lower left")

## Changes for each iteration

In [ ]:
plt.figure()
yy = lowess.lowess_pixel(y, x, frac=1 / 3, n_iter=1)
for nn in range(2, 5):
    yprev = yy
    yy = lowess.lowess_pixel(y, x, frac=1 / 3, n_iter=nn)
    plt.plot(yy - yprev, label=f"{nn} - {nn - 1}")
plt.legend()

## Compare Triangle to LOWESS

### Do the whole image

In [ ]:
# dn_xr

In [ ]:
%%time
x2 = lowess.date2num(dn_xr.date.values)
# dn_xr_lowess = lowess.lowess_xr(dn_xr, frac=1/3) #, min_days_weighted=365.0)
dn_lowess = lowess.lowess_stack(
    # dn_xr.data, x2, frac=1 / 3
    dn_xr_noisy.data, x2, frac=1 / 3
)  # , min_days_weighted=365.0)

In [ ]:
dn_lowess = dn_lowess - dn_lowess[0][None, :, :]
dn_xr_lowess = dn_xr.copy()
dn_xr_lowess.data = dn_lowess
dn_xr_lowess

In [ ]:
# dn_xr_lowess[:, nx//2, ny//2].plot()
# fig, axes = dn_xr_lowess[len(x)//2].plot.imshow()

In [ ]:
plotting.plot_img_diff(arrays=(dn_xr[len(x) // 2], dn_xr_lowess[len(x) // 2]))

In [ ]:
from scipy import ndimage

mode = "reflect"
tri_win = signal.windows.triang(int(1 / 3 * len(x2)))

dn_tri = ndimage.convolve1d(dn_xr_noisy.data, tri_win, axis=0, mode=mode) / sum(tri_win)
dn_tri = dn_tri - dn_tri[0][None, :, :]
dn_xr_tri = dn_xr.copy()
dn_xr_tri.data = dn_tri
# dn_xr_tri

In [ ]:
fig, ax = pplt.subplots()

dn_xr_lowess[:, nx // 2, ny // 2].plot(ax=ax)
dn_xr_tri[:, nx // 2, ny // 2].plot(ax=ax)

In [ ]:
iidx = len(x) // 2
iidx = 62
plotting.plot_img_diff(
    arrays=(
        dn_xr_tri[iidx],
        dn_xr_lowess[iidx],
        dd_xr[iidx],
    ),
    show_diff=False,
    titles=['Triangle', 'LOWESS', 'Truth (no noise)'],
    vm=3,
)

In [ ]:
# # signal.windows.gaussian
# tri_win = signal.windows.triang(int(1 / 3 * len(x)))
# ts_tri = signal.convolve(ts_noisy, tri_win, mode="same") / sum(tri_win)
# ts_tri -= ts_tri[0]


# fig, ax = plt.subplots()

# ax.plot(ts_noisy.date, ts_tri, lw=4, label="", marker=".")


# ax.set_title("Triangle-window smoothing")
# ax.grid()
# ax.legend()

### attempt the mean thing for removing day1

In [ ]:
diffs = dn_xr.data - dn_xr.data[0][None, :, :]
diffs.shape
fig, axes = pplt.subplots(ncols=3)

day1noise_est = -1 * diffs.mean(axis=0)
axes.format(title=["Est", "Truth", "Left - middle"])

ax = axes[0]
axim = ax.imshow(day1noise_est)
ax.colorbar(axim, loc="r")

ax = axes[1]
axim = ax.imshow(day1noise)
ax.colorbar(axim, loc="r")


ax = axes[2]
axim = ax.imshow(day1noise_est - day1noise)
ax.colorbar(axim, loc="r")

s = (150, 150)
print(day1noise_est[s], day1noise[s])

### Plot

In [ ]:
fig, axes = pplt.subplots(figsize=(3, 3))

y = ts_noisy.data
# y_iter1 = lowess.lowess_pixel(y, x, frac=1 / 3, n_iter=1)
y_iter2 = lowess.lowess_pixel(y, x, frac=1 / 3, n_iter=2)
y_iter2_zeroed = y_iter2 - y_iter2[0]

tri_win = signal.windows.triang(int(1 / 3 * len(x)))
# Add the estimate of the day1 noise
ts_noisy_remove_est = ts_noisy.copy() + day1noise_est[s]
ts_noisy_remove_est[0] = 0

# To skip the "remove the average ifg phase"
# ts_noisy_remove_est = ts_noisy.copy()

ts_tri = signal.convolve(ts_noisy_remove_est, tri_win, mode="same") / sum(tri_win)
ts_tri_zeroed = ts_tri - ts_tri[0]


ts_arr = [ts_truth, ts_noisy, y_iter2_zeroed, ts_tri]
# labels = ['Noisy', 'Truth']
labels = ["$\mathbf{d}$", "$\mathbf{\phi}$", "$\mathbf{\hat{\phi}}$", "Triangle"]
colors = ["C1", "C0", "C2", "C3"]


ax = axes[0]
dts = ts_noisy.date.values
# for tt, label, color in zip(ts_arr, labels, colors):
ax.plot(dts, ts_truth, lw=3, label=labels[0], color=colors[0])
ax.plot(dts, ts_noisy, lw=1.5, label=labels[1], marker=".", color=colors[1])
ax.plot(dts, y_iter2_zeroed, lw=3, label=labels[2], color=colors[2])
ax.plot(dts, ts_tri_zeroed, lw=3, label=labels[3], color=colors[3])
# ax.plot(dts, ts_tri, lw=3)

ax.set_ylabel("[cm]")
ax.legend(loc="ur", ncols=2)

# ylim = ax.get_ylim() # Keep the same limits, but make a vert line for whole span
# ax.vlines(img_date, ylim[0], ylim[1], linestyle='--', color='gray', alpha=.5)
# ax.set_ylim(ylim)

In [ ]:
fig.savefig("../figures/chapter5-lowess/figure4-compare-tri.pdf")

In [ ]:
s

In [ ]:
# plt.figure()
# # plt.plot((dn_xr.data + day1noise_est[None, :, :])[:, 150, 150])
# plt.plot((dn_xr.data)[:, 150, 150])

# Filter Comparisons and Frequency Responses

In [ ]:
fig, ax = plt.subplots(figsize=(7, 4))
ts_truth.plot(ax=ax, label="Truth")
ts_noisy.plot(ax=ax, label="Noisy")

## Frequency content of input time series

In [ ]:
x = lowess.date2num(dn_xr.date.values)
x = (x - x[0]) / 365.25
# x

In [ ]:
def plot_fft(ts, x=None, ax=None, label=None):
    if ax is None:
        fig, ax = plt.subplots(figsize=(6, 6))

    # ts_hat = fft(ts)
    # f = fftfreq(len(ts), d=(x[1] - x[0]))
    # ax.plot(fftshift(f), 20*np.log10(fftshift(ts_hat)), label=label)

    Fs = 1 / (x[1] - x[0]) if x is not None else None
    ax.magnitude_spectrum(ts, Fs=Fs, scale="dB", pad_to=512, label=label)
    # ax.set_ylabel("dB")
    ax.set_xlabel("cycles / year")
    ax.set_ylim((-100, None))
    ax.grid(True)
    return ax

In [ ]:
fig, ax = plt.subplots(figsize=(8, 5))

plot_fft(ts_noisy, x, ax=ax, label="Noisy")
plot_fft(ts_truth, x, ax=ax, label="Truth")
ax.legend()

In [ ]:
x = lowess.date2num(dn_xr.date.values)
y = ts_noisy.values
y -= y[0]

## Show frequency response of Triangle window smoothing

In [ ]:
# signal.windows.gaussian
tri_20 = signal.windows.triang(int(0.2 * len(x)))
tri_30 = signal.windows.triang(int(0.3 * len(x)))
tri_40 = signal.windows.triang(int(0.4 * len(x)))
tri_50 = signal.windows.triang(int(0.5 * len(x)))
tri_70 = signal.windows.triang(int(0.7 * len(x)))

ts_tri_20 = signal.convolve(ts_noisy, tri_20, mode="same") / sum(tri_20)
ts_tri_50 = signal.convolve(ts_noisy, tri_50, mode="same") / sum(tri_50)
ts_tri_70 = signal.convolve(ts_noisy, tri_70, mode="same") / sum(tri_70)


for tt in [ts_tri_20, ts_tri_50, ts_tri_70]:
    tt -= tt[0]


ts_arr = [ts_noisy, ts_truth, ts_tri_20, ts_tri_50, ts_tri_70]
labels = ["Noisy", "Truth", "20%", "50%", "70%"]


fig, ax = plt.subplots(1, 1, figsize=(8, 4), sharey=True, sharex=True)


for tt, label in zip(ts_arr, labels):
    ax.plot(ts_noisy.date, tt, lw=4, label=label, marker=".")


ax.set_title("Triangle-window smoothing")
ax.grid()
ax.legend()

## Triangle Window Frequency Response

In [ ]:
fig, ax = plt.subplots(figsize=(8, 5))


for tt, label in zip(ts_arr, labels):
    plot_fft(tt, x, ax=ax, label=label)

ax.legend()

## Compare LOWESS Iterations

In [ ]:
ts_20 = lowess.lowess_pixel(y, x, frac=0.2, n_iter=1)
ts_20_2 = lowess.lowess_pixel(y, x, frac=0.2, n_iter=2)
ts_20_3 = lowess.lowess_pixel(y, x, frac=0.2, n_iter=3)

ts_50 = lowess.lowess_pixel(y, x, frac=0.5)
ts_70 = lowess.lowess_pixel(y, x, frac=0.7)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(8, 4), sharey=True, sharex=True)


ts_noisy.plot(lw=3, marker=".", ax=ax)

ax.plot(ts_noisy.date, ts_20, lw=4, label=f"1 iter")
ax.plot(ts_noisy.date, ts_20_2, lw=4, label=f"2 iter")
ax.plot(ts_noisy.date, ts_20_3, lw=4, label=f"3 iter")

ax.set_title("LOWESS using 20% of data")
ax.grid()
ax.legend()

In [ ]:
for tt in [ts_20, ts_20_2, ts_20_3, ts_50, ts_70]:
    tt -= tt[0]

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(8, 4), sharey=True, sharex=True)


ts_noisy.plot(lw=3, marker=".", ax=ax)
ts_truth.plot(lw=2, marker=".", ax=ax)

ax.plot(ts_noisy.date, ts_20, lw=4, label=f"1 iter")
ax.plot(ts_noisy.date, ts_20_2, lw=4, label=f"2 iter")
ax.plot(ts_noisy.date, ts_20_3, lw=4, label=f"3 iter")

ax.set_title("LOWESS using 20% of data")
ax.grid()
ax.legend()

## Compare LOWESS window sizes

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(8, 4), sharey=True, sharex=True)


ts_noisy.plot(lw=3, marker=".", ax=ax)
ts_truth.plot(lw=2, marker=".", ax=ax)

ax.plot(ts_noisy.date, ts_20, lw=4, label="20%")
ax.plot(ts_noisy.date, ts_50, lw=2.2, label=f"50%")
ax.plot(ts_noisy.date, ts_70, lw=1.6, label=f"70%")

ax.grid()
ax.legend()

# ax.set_xlim([start_date - pd.offsets.Day(30), end_date + pd.offsets.Day(30)])
# ax.set_ylim((-8, 6))
# ax.set_ylim((-1, 12))

In [ ]:
fig, ax = plt.subplots(figsize=(8, 5))

ts_hat = fft(ts_noisy)
f = fftfreq(len(ts_noisy), d=(x[1] - x[0]))


labels = ["Noisy", "Truth", "20%", "50%", "70%"]
ts_arr = [ts_noisy, ts_truth, ts_20, ts_50, ts_70]
for tt, label in zip(ts_arr, labels):

    # ax.plot(fftshift(f), 20*np.log10(fftshift(fft(tt))), label=label, lw=3)
    plot_fft(tt, x, ax=ax, label=label)

    #
    ax.legend()
# ax.set_ylabel("dB")
ax.set_xlabel("cycles / year")

# Dealing with big spikes

In [ ]:
ts_noisy_spiked = ts_noisy.copy()

p = 0.05  # probability of spikes
signs = RNG.choice([-1, 1], size=len(ts_noisy))
# amp = 8
amp = 5
spike_idxs = RNG.uniform(size=len(ts_noisy)) < p
spike_idxs = np.where(spike_idxs)[0]
print(f"Adding {sum(spike_idxs)} spike noises of size {amp}")
ts_noisy_spiked[spike_idxs] += signs[spike_idxs] * amp
ts_noisy_spiked[spike_idxs] += signs[spike_idxs + 1] * 0.5 * amp

In [ ]:
ts_noisy_spiked.plot()

In [ ]:
y = ts_noisy_spiked.values
y -= y[0]

# ts_tri_spike = signal.convolve(y, tri_50, mode='same') / sum(tri_50)
# ts_spike = lowess.lowess_pixel(y, x, frac=0.5)
ts_tri_spike = signal.convolve(y, tri_40, mode="same") / sum(tri_40)
ts_spike = lowess.lowess_pixel(y, x, frac=0.4)
# ts_tri_spike = signal.convolve(y, tri_30, mode='same') / sum(tri_30)
# ts_spike = lowess.lowess_pixel(y, x, frac=0.3)

ts_tri_spike -= ts_tri_spike[0]
ts_spike -= ts_spike[0]

In [ ]:
ts_arr = [ts_noisy_spiked, ts_truth, ts_tri_spike, ts_spike]
labels = ["Noisy", "Truth", "Triangle", "LOWESS"]


fig, ax = plt.subplots(1, 1, figsize=(8, 4), sharey=True, sharex=True)


for tt, label in zip(ts_arr, labels):
    ax.plot(ts_noisy.date, tt, lw=4, label=label, marker=".")


ax.set_title("Triangle vs. LOWESS with outliers")
ax.set_ylim((-2.5, 4))
ax.grid()
ax.legend(loc="lower right")

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(10, 5))
ax = axes[0]

labels = ["Orig Noisy", "Spiked Noisy", "Truth", "Triangle", "LOWESS"]

ts_arr = [ts_noisy, ts_noisy_spiked, ts_truth, ts_tri_spike, ts_spike]
for tt, label in zip(ts_arr, labels):

    # ax.plot(fftshift(f), 20*np.log10(fftshift(fft(tt))), label=label, lw=3)
    plot_fft(tt, x, ax=ax, label=label)
    plot_fft(tt, x, ax=axes[1], label=label)


for ax in axes:
    ax.legend()
    # ax.set_ylabel("dB")
    ax.set_xlabel("cycles / year")

axes[1].set_ylim((-50, None))
axes[1].set_xlim((None, 6))

# Tracking sinusoids

In [ ]:
f_years = 1
plt.figure(figsize=(9, 3))
plt.plot(ts_noisy.date, np.sin(2 * np.pi * f_years * x))

In [ ]:
ts_noisy_seasonal = ts_noisy.copy()
yearly_amp = 2
semi_amp = 1
seasonal_signal = yearly_amp * np.sin(2 * np.pi * 1.0 * x) + semi_amp * np.sin(
    2 * np.pi * 2 * x
)
ts_noisy_seasonal += seasonal_signal
ts_truth_seasonal = ts_truth.copy() + seasonal_signal

fig, ax = plt.subplots()
ts_noisy_seasonal.plot(ax=ax, lw=2)
ts_truth_seasonal.plot(ax=ax, lw=2)
# ax.plot(ts_noisy_seasonal.date, seasonal_signal)
ax.set_ylabel("[cm]")
ax.set_title("Noisy signal with annual/semi-annual")

In [ ]:
fig, ax = plt.subplots(figsize=(7, 4))
plot_fft(ts_noisy, x, label="Original", ax=ax)
plot_fft(ts_noisy_seasonal, x, label="seasonal", ax=ax)
ax.legend()

In [ ]:
y = ts_noisy_seasonal.data
ts_20 = lowess.lowess_pixel(y, x, frac=0.2)
ts_50 = lowess.lowess_pixel(y, x, frac=0.5)
ts_70 = lowess.lowess_pixel(y, x, frac=0.7)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(8, 4), sharey=True, sharex=True)

ts_noisy_seasonal.plot(lw=3, marker=".", ax=ax, label="noisy")
ts_truth_seasonal.plot(lw=2, marker=".", ax=ax, label="truth")
ax.plot(ts_noisy.date, ts_20, lw=4, label="20%")
ax.plot(ts_noisy.date, ts_50, lw=2.2, label=f"50%")
ax.plot(ts_noisy.date, ts_70, lw=1.6, label=f"70%")

ax.grid()
ax.legend()

# ax.set_xlim([start_date - pd.offsets.Day(30), end_date + pd.offsets.Day(30)])
# ax.set_ylim((-8, 6))
# ax.set_ylim((-1, 12))

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(8, 4), sharey=True, sharex=True)


ts_noisy_seasonal.plot(lw=3, marker=".", ax=ax, label="noisy")
ts_truth_seasonal.plot(lw=2, marker=".", ax=ax, label="truth")


ax.plot(ts_noisy.date, ts_20, lw=4, label="20%")
ax.plot(ts_noisy.date, ts_50, lw=2.2, label=f"50%")
ax.plot(ts_noisy.date, ts_70, lw=1.6, label=f"70%")

ax.grid()
ax.legend()

# ax.set_xlim([start_date - pd.offsets.Day(30), end_date + pd.offsets.Day(30)])
# ax.set_ylim((-8, 6))
# ax.set_ylim((-1, 12))